<a href="https://colab.research.google.com/github/NK-kimiya/natural_language_processing/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTモデルまとめ（Self-Attention理解用）

## ✅ BERTモデルとは
- TransformerのEncoder部分だけを用いた自然言語モデル

### 特徴
1. 文の前後関係を同時に理解できる（双方向）

## ✅ モデルの全体構造

**STEP1：入力トークン列**  
- 例："私はりんごが好きです"

**STEP2：トークンID・埋め込み**  
- 入力をトークンに分割して、それぞれをベクトル（例：768次元）に変換

**STEP3：Transformer Encoder × N層**  
- 各層で Self-Attention + FeedForward を実行

BERT内部の FeedForward は、「各トークンのベクトル」に対して個別に処理

Flattenせず、トークンごとのベクトル構造（行列）は維持しながら全結合に通す

**STEP4：トークンごとの出力ベクトルを取得**

## ✅ Self-Attentionとは？
- 各単語が、他の単語とどれくらい関係があるか（注目すべきか）を学ぶ層

## ✅ Self-Attentionの流れ（5トークン × 768次元で例）

### ⑴ 入力行列
X.shape = (5, 768) # 5トークン分の埋め込みベクトル
<br>
### ⑵ 重み行列による変換

#### ① Query（問い）を作る重み行列
- 「注目したい単語（何が気になるか）」を表す

　　　W_Q: (768, 64) Q = X @ W_Q → (5, 64)
　　　※(5トークン , 埋め込みの次元を64次元に圧縮)

#### ② Key（鍵）を作る重み行列
- 「自分がどう注目されるべきか（どんな特徴を持つか）」を表す

　　　W_K: (768, 64) K = X @ W_K → (5, 64)
　　　※(5トークン , 埋め込みの次元を64次元に圧縮)

#### ③ Value（情報）を作る重み行列
- 単語が実際に持っている意味的情報を表す

　　　W_V: (768, 64) V = X @ W_V → (5, 64)
　　　※(5トークン , 埋め込みの次元を64次元に圧縮)

### ⑶ QとKの内積 → 注目スコアの計算
- 内積によって各トークン間の関係性（類似度）を算出

　　　AttentionScore = Q @ K.T → (5, 5)

#### 例：Qの1トークン目とKの全トークンの内積（スカラー）を計算
- Q₁・K₁ → スカラー①  
- Q₁・K₂ → スカラー②  
- Q₁・K₃ → スカラー③  
- Q₁・K₄ → スカラー④  
- Q₁・K₅ → スカラー⑤

→ これをQ₂〜Q₅についても計算することでトークン同士の関係性が計算できる → 最終的に (5, 5) のスコア行列に

---

### ⑷ Softmaxでスコアを正規化（行方向に）

AttentionWeight = softmax(AttentionScore / √64) → (5, 5)
<br>
### ⑸ 正規化されたスコアを用いて情報を合成
Output = AttentionWeight @ V → (5, 64)

→ 各トークンが文脈を反映した出力ベクトルを持つようになる

### ⑹ BERTの出力を全結合層へ渡す
Output（5, 64）→ Flatten or [CLS]抽出 → 全結合層
※BERTでは入力の先頭に [CLS]（Classification）という特別なトークンが追加

※文分類・感情分析・質問応答などのタスクでは、文全体の意味が必要

※そこで、「[CLS]の出力ベクトル（例：768次元）」を使えば、文の代表ベクトルとして使える！
## ✅ 学習で更新される重み

1. Transformer内部の重み：  
   - W_Q, W_K, W_V  
   - 全結合層の重み  
2. 出力層（タスクごとの分類器など）の全結合層の重み






## メモ

✔ 全結合層（Linear）は、1次元ベクトルを前提としているため、
入力が行列や3次元配列（画像など）の場合、
✅ Flattenしてベクトル化するか、ベクトル単位に分割して処理する必要がある。

✔ 一方、Transformerは 行列を維持しながら計算しているように見えるが、
実際は「各トークン（＝1ベクトル）ごと」に全結合を個別に適用しているだけなので、
✅ 本質的には「ベクトルにLinearをかけている」に変わりはない。

画像データもtransformerで学習できる。

画像を小さな領域(パッチ)に分割

例えば、224×224ピクセルの画像を16×16のパッチに分けると、196個のパッチができる。　各パッチは画像の (チャネル数, 高さ, 幅) を持つ3次元配列なので、
それを 1次元ベクトルにFlattenすると、自然言語のトークンのように扱える
